# Data Pre-Processing

In [97]:
import json
import pandas as pd

from pathlib import Path

In [98]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_dataset_pg.csv", index_col=0)

In [99]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/comics_ZSL/scripts/scene_discriptions_vision.json", "r") as f:
    data = json.load(f)

In [100]:
new_column = data

In [101]:
df['scene_description'] = new_column

In [102]:
df

,file_name,page_nr,split,utterance,emotion_c,scene_description
0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",The comic page depicts a dramatic confrontatio...
1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",The comic page depicts a tense and intense sce...
2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",The image depicts a comic book page featuring ...
3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",The comic page depicts a dynamic scene with tw...
4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",This comic page depicts a dramatic confrontati...
...,...,...,...,...,...,...
869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",The comic page presents a dramatic scene with ...
870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",The comic page depicts a tense scene where a g...
871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",The comic page presents a narrative that unfol...
872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",The comic page depicts a tense scene where a f...


In [115]:
df['utterance'] = df['utterance'].apply(json.loads)

### Prepare prompts

In [116]:
# Formatting Fx
# Build questoin
# Build answer

In [117]:
def formatting_fct(instruction="", input="", output=""):
    
    prompt_d ={
            
        "instruction": f"""{instruction}""",
        "input": f"""{input}""",
        "output": f"""{output}"""
            
        }
    
    return prompt_d

In [118]:
# def build_instruction():
#     emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

# INPUT:
# - You will receive a list of utterances from a page in a comic book
# - The utterance may express one or multiple emotions

# TASK:
# 1. Carefully analyze the emotional context and tone of each utterance in the page
# 2. Identify applicable emotions from the following classes:
#    {formatted_classes}
# 3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

# RULES:
# 1. Use ONLY the labels listed above
# 2. Output must be a JSON with single key "page_utterance_emotions"
# 3. Value must be an array where:
#    - Each element is an array of emotions for one utterance
#    - Order matches the input utterances order
#    - Multiple emotions are allowed per utterance
# 4. No explanations, only JSON output

# IMPORTANT:
# - Each array element corresponds to one utterance
# - One utterance can have multiple emotions
# - Maintain exact spelling and case of emotion labels
# - Keep emotions in arrays even for single emotions

# """
#     return instruction

In [119]:
def build_instruction():
    emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

INPUT:
- You will receive a list of utterances from a page in a comic book
- The utterance may express one or multiple emotions

TASK:
1. Carefully analyze the emotional context and tone of each utterance in the page
2. Identify applicable emotions from the following classes:
   {formatted_classes}
3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

RULES:
1. Use ONLY the labels listed above
2. Output MUST BE a one-line compact JSON with single key "page_utterance_emotions"
3. Respond ONLY with the JSON object. No additional text before or after.
4. Value must be an array where:
   - Each element is an array of emotions for one utterance
   - Order matches the input utterances order
   - Multiple emotions are allowed per utterance
5. No explanations, only JSON output

IMPORTANT:
- Respond with a ONE-LINE JSON
- Each array element corresponds to one utterance
- One utterance can have multiple emotions
- Maintain exact spelling and case of emotion labels
- Keep emotions in arrays even for single emotions

"""
    return instruction

In [120]:
# def build_tagged_text(utterances):

#     concatenated_utterances = '\n'.join(f"{i + 1}. {line}" for i, line in enumerate(utterances))
    
#     question = f"""Now analyze these utterances in a page:\n{concatenated_utterances}"""

#     return question

In [121]:
def build_tagged_text(scene_description, utterances):
    # Format the utterances with numbering
    utterance_list = '\n'.join(f'{i+1}: {line}' for i, line in enumerate(utterances))
    
    # Create the question with both scene description and utterances
    question = f"""Scene Description: {scene_description}

Utterances:
{utterance_list}"""

    return question

In [122]:
def build_answer(utterance_emotions):
                

    return json.dumps({"emotions": utterance_emotions})

### Build Data Files

In [123]:
df_train = df[df.split == 'TRAIN'].reset_index()

data_file_train = []

for index, _ in df_train.iterrows():
    
    i = index

    instruction = build_instruction()
    question = build_tagged_text(df_train.iloc[i].scene_description, df_train.iloc[i].utterance)
    answer = build_answer(df_train.iloc[i].emotion_c)
    
    data_file_train.append( formatting_fct(instruction, question, answer) )

In [124]:
len(data_file_train)

718

In [125]:
print(data_file_train[0]['instruction'])

### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

INPUT:
- You will receive a list of utterances from a page in a comic book
- The utterance may express one or multiple emotions

TASK:
1. Carefully analyze the emotional context and tone of each utterance in the page
2. Identify applicable emotions from the following classes:
   "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"
3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

RULES:
1. Use ONLY the labels listed above
2. Output MUST BE a one-line compact JSON with single key "page_utterance_emotions"
3. Respond ONLY with the JSON object. No additional text before or after.
4. Value must be an array where:
   - Each element is an array of emotions for one utterance
   - Order matches the input utteran

In [126]:
print(data_file_train[0]['input'])

Scene Description: The comic page depicts a dramatic confrontation between two characters, with the villainous figure, Matilda, at the center. Her facial expression is twisted in a snarl, showcasing a mix of anger and menace, while her body language is tense and aggressive, with her arms raised and fists clenched. Her micro-expressions reveal a deep-seated hatred and resentment towards the protagonist.

In contrast, the protagonist's expression is one of determination and fear, with their eyes wide and mouth open in a scream. Their body language is defensive, with their arms raised to protect themselves from Matilda's attack. Their micro-expressions convey a sense of desperation and urgency.

The emotional atmosphere of the scene is tense and foreboding, with the dark colors and dramatic lighting contributing to a sense of foreboding. The narrative text provides context to the emotional state, highlighting the protagonist's fear and determination to protect themselves from Matilda's at

In [127]:
print(data_file_train[0]['output'])

{"emotions": "[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]"}


In [134]:
df_test = df[df.split == 'TEST'].reset_index()

data_file_test = []

for index, _ in df_test.iterrows():
    
    i = index

    instruction = build_instruction()
    question = build_tagged_text(df_test.iloc[i].scene_description, df_test.iloc[i].utterance)
    answer = build_answer(df_test.iloc[i].emotion_c)
    
    data_file_test.append( formatting_fct(instruction, question, answer) )

In [135]:
len(df_test)

156

In [140]:
print(data_file_test[3]['input'])

Scene Description: The comic page presents a tense scene between a police officer and a young man, with the officer's stern expression contrasting with the young man's nervous demeanor.

**Facial Expressions and Body Language**

* The police officer's face is stern, with a furrowed brow and pursed lips, conveying a sense of authority and warning.
* The young man's face is nervous, with a wide-eyed expression and a slight sweat on his forehead, indicating anxiety and fear.

**Emotional Atmosphere**

* The scene is tense and serious, with the police officer's stern expression setting the tone.
* The young man's nervousness is palpable, as he is clearly intimidated by the officer's presence.

**Emotional Subtext and Contrast**

* The contrast between the officer's stern expression and the young man's nervousness creates an emotional tension between the two characters.
* The officer's words, "Call me 'Lady Cop', zero tolerance, okay?" suggest a strict and no-nonsense approach, which adds t

In [141]:
print(data_file_test[3]['output'])

{"emotions": "[['anger', 'disgust'], ['anger', 'disgust'], ['anger', 'disgust'], ['fear', 'sadness'], ['fear', 'sadness', 'surprise'], ['sadness'], ['sadness'], ['fear', 'sadness'], ['sadness', 'surprise'], ['sadness', 'surprise'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger', 'disgust'], ['joy'], ['joy'], ['surprise', 'joy'], ['surprise', 'joy'], ['anger', 'surprise'], ['anger', 'surprise']]"}


### Create and save JSON files

In [142]:
file_path = Path("/Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/datasets") / "comics35_utterance_pg_scene_train.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_train, file)

In [143]:
file_path = Path("/Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/datasets") / "comics35_utterance_pg_scene_test.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_test, file)